In [1]:
# install dependencies


In [2]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split



In [3]:
# load and create labeled data
data_dir = '/kaggle/input/mangofruitdds/MangoFruitDDS/SenMangoFruitDDS_original'
batch_size = 32
epochs = 20
input_shape = (224, 224, 3)
image_paths = []
labels = []

for category in os.listdir(data_dir):
    category_dir = os.path.join(data_dir, category)
    if os.path.isdir(category_dir):
        for image_filename in os.listdir(category_dir):
            if image_filename.endswith('.jpg'):
                image_path = os.path.join(category_dir, image_filename)
                image_paths.append(image_path)
                labels.append(category)


In [4]:
# split dataset
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

In [5]:
# create data generator for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': train_image_paths, 'label': train_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 689 validated image filenames belonging to 5 classes.


In [6]:
# build model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # 5 classes (4 diseases + 1 healthy)


In [7]:
# compile 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [8]:
# check point
checkpoint = ModelCheckpoint('mango_fruit_disease_model.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

In [9]:
# train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/20
21/21 [==============================] - 35s 2s/step - loss: 2.2491 - accuracy: 0.2557
Epoch 2/20
21/21 [==============================] - 33s 2s/step - loss: 1.3306 - accuracy: 0.3881
Epoch 3/20
21/21 [==============================] - 33s 2s/step - loss: 1.1787 - accuracy: 0.4810
Epoch 4/20
21/21 [==============================] - 33s 2s/step - loss: 1.1290 - accuracy: 0.5053
Epoch 5/20
21/21 [==============================] - 34s 2s/step - loss: 0.9627 - accuracy: 0.6134
Epoch 6/20
21/21 [==============================] - 33s 2s/step - loss: 0.9451 - accuracy: 0.5906
Epoch 7/20
21/21 [==============================] - 34s 2s/step - loss: 0.8948 - accuracy: 0.6027
Epoch 8/20
21/21 [==============================] - 33s 2s/step - loss: 0.8377 - accuracy: 0.6438
Epoch 9/20
21/21 [==============================] - 33s 2s/step - loss: 0.8010 - accuracy: 0.6560
Epoch 10/20
21/21 [==============================] - 34s 2s/step - loss: 0.7925 - accuracy: 0.6591
Epoch 11/20
21/21 [

In [10]:
# save model
model.save('mango_fruit_disease_model.h5')

In [11]:
# evaluate on test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': test_image_paths, 'label': test_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy * 100:.2f}%')

Found 173 validated image filenames belonging to 5 classes.
6/6 [==============================] - 2s 324ms/step - loss: 0.7952 - accuracy: 0.6879
Test loss: 0.7952
Test accuracy: 68.79%


In [12]:
# End